# Segment Vs Kinesis Analysis

In [1]:
import pgdbConnection

In [2]:
conn=pgdbConnection.get_connection()
cursor = conn.cursor()
cursor.execute("SELECT datname from pg_database")
rows = cursor.fetchall()

Connecting to database
  ->host=data-pier-staging.cl8qfdl47mtr.ap-southeast-1.rds.amazonaws.com port=5432 dbname=data_pier user=data_team password=Z1QxYKXO9qjnXQwNVZlmofAWRjMth1nx
Connected!



In [3]:
print ("Show me the databases:")
for row in rows:
    print (row[0])

Show me the databases:
template0
rdsadmin
template1
postgres
c3po
data_pier


In [4]:
cursor.execute("SELECT COUNT(*),CAST(received_at AS DATE) AS rdt FROM moneysmartsg_prod.pages WHERE url LIKE '%moneysmart.sg/%' AND received_at at TIME zone 'utc' at TIME zone 'SGT' >= '2019-12-04' GROUP BY rdt;")
rows = cursor.fetchall()

import pprint
pprint.pprint(rows)

[(109475, datetime.date(2019, 12, 4)),
 (184430, datetime.date(2019, 12, 5)),
 (149887, datetime.date(2019, 12, 6)),
 (140411, datetime.date(2019, 12, 7)),
 (149821, datetime.date(2019, 12, 8)),
 (172917, datetime.date(2019, 12, 9)),
 (191489, datetime.date(2019, 12, 10)),
 (190091, datetime.date(2019, 12, 11)),
 (60763, datetime.date(2019, 12, 12))]


In [5]:
#!pip install pyathena
from pyathena import connect
import pandas as pd
import athenaConnection

In [6]:
athena_conn = connect_to_s3()

NameError: name 'connect_to_s3' is not defined

In [ ]:
print("Checking athena works")
query = "select * from ms_data_stream_production_processed limit 10"
test_query_results = pd.read_sql(query, athena_conn)
test_query_results.head(10)